## Fires Exploration with NASA FIRMS

Notebook for exploring whether fire detections from NASA FIRMS data can be useful for detecting TPA landfill sites.

In [18]:
import folium
import geopandas as gpd
import glob
import os
import pandas as pd

### Load in data with GeoPandas

Note that you will need to pull down the fires data from Indonesia to get this to run.
GDrive link: https://drive.google.com/file/d/16kGZy1JMky2mO_rtjqLOFevBXZay8uYk/view?usp=sharing

In [2]:
data_dir = '../data'

In [3]:
fires_file = os.path.join(data_dir, 'fires_v2', 'bali_fires.geojson')
tpa_file = os.path.join(data_dir, 'tpa_points.json')

In [4]:
tpa = gpd.read_file(tpa_file)
tpa.head(5)

,id,From Kab,From_#KEC,ID,Kab,Kecamatan,Name,Surface_Ha,TOT_Kg/Day,geometry
0,00000000000000000004,NaN,NaN,5,Klungkung,Nusa Penida,TPA Jungut Batu,1.20,NaN,POINT (115.45941 -8.67096)
1,00000000000000000005,1.0,1.0,6,Klungkung,Nusa Penida,TPA Biaung,1.85,9433.0,POINT (115.49802 -8.67993)
2,00000000000000000003,1.0,3.0,4,Klungkung,Dawan,TPA Sente,1.00,43219.0,POINT (115.45446 -8.53037)
3,00000000000000000006,1.0,4.0,7,Bangli,Bangli,TPA Regional Bangli,0.99,47350.0,POINT (115.36793 -8.35354)
4,00000000000000000008,1.0,5.0,9,Jembrana,Negara,TPA Peh,2.00,38130.0,POINT (114.58329 -8.32794)


In [5]:
print('There are {} TPA sites'.format(len(tpa)))

There are 10 TPA sites


In [6]:
fires = gpd.read_file(fires_file)
fires.head(5)

,acq_date,acq_time,daynight,version,track,bright_ti5,bright_ti4,scan,confidence,type,instrument,frp,satellite,geometry
0,2015-01-04,1740,N,1,0.37,246.32,309.68,0.40,n,0,VIIRS,1.58,N,POINT (115.20852 -8.64300)
1,2015-04-03,0524,D,1,0.53,288.85,327.30,0.59,n,0,VIIRS,4.14,N,POINT (115.09514 -8.53138)
2,2015-07-01,0555,D,1,0.36,296.35,331.29,0.38,n,0,VIIRS,2.80,N,POINT (115.05491 -8.27026)
3,2015-07-07,0542,D,1,0.39,299.03,330.73,0.45,n,0,VIIRS,1.51,N,POINT (115.21693 -8.80463)
4,2015-07-12,0548,D,1,0.36,299.94,331.85,0.39,n,3,VIIRS,2.67,N,POINT (115.46450 -8.70497)


In [7]:
print('There are {} fires from Jan 1 2015 over Bali'.format(len(fires)))

There are 1789 fires from Jan 1 2015 over Bali


In [8]:
# write out bali fires
fires.to_file(os.path.join(data_dir, 'fires_v2', 'bali_fires.shp')) # for storymap

### Do fires fall within TPA sites?

In [9]:
# buffer TPA points into polygons
buffer_width = 0.01 # degrees
tpa['geometry'] = tpa.set_crs(epsg=4326).buffer(buffer_width).envelope

<ipython-input-9-4b367d6d651b>:3: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  tpa['geometry'] = tpa.set_crs(epsg=4326).buffer(buffer_width).envelope


In [10]:
# join fires that fall within the bounding boxes of the TPA sites
fires_tpa = gpd.sjoin(fires, tpa, how='inner', op='intersects')

In [11]:
# only keep fire detections with high confidence (marked nominal "n")
fires_tpa = fires_tpa[fires_tpa['confidence'] == 'n']

In [12]:
fires_tpa.head(5)

,acq_date,acq_time,daynight,version,track,bright_ti5,bright_ti4,scan,confidence,type,...,index_right,id,From Kab,From_#KEC,ID,Kab,Kecamatan,Name,Surface_Ha,TOT_Kg/Day
1,2015-04-03,0524,D,1,0.53,288.85,327.30,0.59,n,0,...,8,00000000000000000002,1.0,10.0,3,Tabanan,Kerambitan,TPA Mandung,2.50,64580.0
112,2015-11-27,1704,N,1,0.59,289.42,311.90,0.38,n,0,...,6,00000000000000000007,1.0,7.0,8,Buleleng,Kubutambahan,TPA Bengkala,0.35,125350.0
113,2015-11-27,1704,N,1,0.59,289.45,304.94,0.38,n,0,...,6,00000000000000000007,1.0,7.0,8,Buleleng,Kubutambahan,TPA Bengkala,0.35,125350.0
114,2015-11-27,1704,N,1,0.59,289.96,319.25,0.38,n,0,...,6,00000000000000000007,1.0,7.0,8,Buleleng,Kubutambahan,TPA Bengkala,0.35,125350.0
1605,2019-10-19,1739,N,1,0.36,287.11,338.31,0.38,n,0,...,6,00000000000000000007,1.0,7.0,8,Buleleng,Kubutambahan,TPA Bengkala,0.35,125350.0


In [13]:
print('Found {} fires that fall within TPA sites'.format(len(fires_tpa)))

Found 17 fires that fall within TPA sites


In [19]:
unique_sites = pd.unique(fires_tpa['Name'])

In [21]:
print('There are {} unique sites with fires'.format(len(unique_sites)))
print(unique_sites)

There are 5 unique sites with fires
['TPA Mandung' 'TPA Bengkala' 'TPA Regional Suwung' 'TPA Biaung'
 'TPA Temesi']


### Visualize fires on a Folium map

In [24]:
# create a folium map centered on Bali
# use Google satellite hybrid view as our basemap
m = folium.Map(location=(-8.4199969,114.9164863), 
               tiles='https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}', 
               attr='Google',
               name='Google Satellite Hybrid')

# add TPA bounding boxes
tpa_locs = folium.GeoJson(tpa, name='TPA Locations').add_to(m)

# add Bali fires
def add_circle(f):
    lon, lat = f['geometry'].x, f['geometry'].y
    circle = folium.CircleMarker(location=(lat, lon), 
                                 radius=2, 
                                 weight=f['frp']/5, 
                                 color='red',
                                 fillColor='red',
                                 opacity=0.8,
                                 fillOpacity=0.8).add_to(m)
    
for ridx, row in fires.iterrows():
    add_circle(row)

popup = folium.LatLngPopup().add_to(m)

In [25]:
display(m)

In [16]:
# save map as HTML
m.save('../maps/FIRMS_fires_bali.html')